Najprej bomo prevedli celoštevilski linearni program iz navodil v funkcijo `alpha_od(G)`

In [7]:
def alpha_od(G):
    """
    Compute the size of the maximum odd independent set in the graph G.
    """
    from sage.numerical.mip import MixedIntegerLinearProgram
    
    n = G.order()
    V = G.vertices()
    
    # Create the mixed integer linear program
    mip = MixedIntegerLinearProgram(maximization=True)
    
    # Define variables
    x = mip.new_variable(binary=True) # Indicator if vertex v is in the independent set
    y = mip.new_variable(binary=True) # Indicator if vertex v has neighbors in the independent set
    z = mip.new_variable(integer=True) # Counter for the vertex v
    
    # Objective function
    mip.set_objective(mip.sum(x[v] for v in V))
    
    # Constraints
    for u, v in G.edges(labels=False):
        mip.add_constraint(x[u] + x[v] <= 1)
        
    for u in V:
        sum_neighbors = mip.sum(x[v] for v in G.neighbors(u))
        mip.add_constraint(sum_neighbors <= n * y[u])
        mip.add_constraint(y[u] + sum_neighbors == 2*z[u])
    
    # Solve the MIP
    return mip.solve()

Zapisali bomo tudi CLP za določanje najmanjše število barv za krepko liho barvanje grafa $G$.

In [8]:
def chi_so(G):
    """
    Compute the size of the minimum strong odd coloring of the graph G.
    """
    from sage.numerical.mip import MixedIntegerLinearProgram
    
    n = G.order()
    V = G.vertices()
    
    # Create the mixed integer linear program
    mip = MixedIntegerLinearProgram(maximization=False)
    
    # Define variables
    x = mip.new_variable(binary=True) # Indicator if vertex v has color i
    y = mip.new_variable(binary=True) # Indicator if color i is used
    z = mip.new_variable(integer=True) # Counter for vertex v and color i
    w = mip.new_variable(binary=True) # Indicator if color i appears in the neighborhood of vertex v
    
    # Objective function
    mip.set_objective(mip.sum(y[i] for i in range(n))) # n is obvious upper bound on colors
    
    # Constraints
    for v in V:
        mip.add_constraint(mip.sum(x[v, i] for i in range(n)) == 1) # Each vertex gets exactly one color
        
    for u, v in G.edges(labels=False):
        for i in range(n):
            mip.add_constraint(x[u, i] + x[v, i] <= 1) # Adjacent vertices have different colors
            
    for i in range(n):
        mip.add_constraint(mip.sum(x[v, i] for v in V) <= n * y[i]) # If color i is used, y[i] = 1
   
    for v in V:
        for i in range(n):
            sum_neighbors = mip.sum(x[u, i] for u in G.neighbors(v))
            deg = len(G.neighbors(v))
            mip.add_constraint(w[v, i] <= sum_neighbors) # w[v,i] = 0 if color i does not appear in neighborhood of v
            mip.add_constraint(sum_neighbors <= deg * w[v, i]) # w[v,i] = 1 if color i appears in neighborhood of v
            # these two conditions together mean w[v,i] = 1 iff color i appears in neighborhood of v
            mip.add_constraint(w[v, i] + sum_neighbors == 2 * z[v, i]) # Strong odd coloring condition
            mip.add_constraint(z[v, i] >= 0)
    
    # Solve the MIP
    return mip.solve()

Na nekaj grafih preverimo delovanje (samo test)

In [9]:
from sage.all import Graph
from sage.graphs.graph_generators import graphs

P4 = Graph(5)
C4 = Graph([(0,1),(1,2),(2,3),(3,0)])
C5 = Graph([(0,1),(1,2),(2,3),(3,4),(4,0)])

for name, G in [("P4", P4), ("C4", C4), ("C5", C5)]:
    val = alpha_od(G)
    print(name, "alpha_od =", val)

G1 = Graph(5)
G3 = graphs.CompleteBipartiteGraph(2, 3)
for name, G in [("G1", G1), ("G3", G3)]:
    val = chi_so(G)
    print(name, "chi_so =", val)

P4 alpha_od = 5.0
C4 alpha_od = 1.0
C5 alpha_od = 1.0
G1 chi_so = 1.0
G3 chi_so = 3.0


Zapisali bomo tudi funkcijo, ki generira vse grafe dane velikosti $n$ z lastnostjo `alpha_od(G)` = 1.

In [10]:
def generate_graphs_with_alpha_od_equal_to_1(n):
    """
    Iterate over all non-isomorphic graphs with n vertices using nauty_geng and return a set of those graphs G for which alpha_od(G) == 1. It is meant to used for n <= 9.
    """
    from sage.graphs.graph_generators import graphs
    
    matches = list()
    for G in graphs(n):
        if G.diameter() > 2:
            continue  # Neccessary condition for alpha_od(G) == 1 is diameter at most 2
        elif alpha_od(G) == 1:
            matches.append(G)
            
    return matches

# Example usage:
graphs_with_alpha_od_1 = generate_graphs_with_alpha_od_equal_to_1(3)
len(graphs_with_alpha_od_1)
# na mojem računalniku sem potreboval ~40 sekund za n=8

2

We will also need a `is_claw_free(G)` function

In [11]:
def is_claw_free(G):
    """
    Check if the given graph G is claw-free. A graph is claw-free if it does not contain K_{1,3} as an induced subgraph.
    """
    from sage.graphs.graph_generators import graphs
    
    claw = graphs.ClawGraph() # K_{1,3}
    return G.subgraph_search(claw, induced=True) is None # subgraph_search returns None if no such subgraph is found

In [12]:
print(is_claw_free(graphs.CycleGraph(10)))
print(is_claw_free(graphs.PathGraph(7)))
print(is_claw_free(graphs.CompleteGraph(6)))

print(is_claw_free(graphs.StarGraph(4)))
print(is_claw_free(graphs.CompleteBipartiteGraph(3,3)))

True
True
True
False
False


We will also need a function that generates graphs with this `alpha_od(G)` == 1 for  $|V| \geq 10$. We will use a stochastic apporach.

In [13]:
def add_edges_until_diameter_leq_2(G, seed=None):
    """
    Given a graph G, iteratively add edges so that the resulting
    graph H has diameter <= 2.
    
    Process:
    - Compute the diameter path
    - Randomly pick two non-adjacent vertices on that path
    - Add one edge between them
    - Repeat
    
    Returns the modified graph H.
    """
    
    import random
    if seed is not None:
        random.seed(int(seed))

    H = G.copy()
    diameter = H.diameter()

    while diameter > 2:
        # Find all pairs of vertices with distance equal to diameter
        max_distance_pairs = []
        vertices = H.vertices()
        
        for i in range(len(vertices)):
            for j in range(i + 1, len(vertices)):
                u, v = vertices[i], vertices[j]
                if H.distance(u, v) == diameter:
                    max_distance_pairs.append((u, v))
        
        if not max_distance_pairs:
            break
            
        # Pick a random pair at maximum distance
        u, v = random.choice(max_distance_pairs)
        
        # Get the shortest path between them
        path = H.shortest_path(u, v)
        
        # Find non-adjacent vertex pairs in the path
        non_adjacent_pairs = []
        for i in range(len(path)):
            for j in range(i + 2, len(path)):  # i+2 ensures non-adjacent in path
                a, b = path[i], path[j]
                if not H.has_edge(a, b):
                    non_adjacent_pairs.append((a, b))
        
        if not non_adjacent_pairs:
            # Remove this pair and try another
            max_distance_pairs.remove((u, v))
            if max_distance_pairs:
                continue
            else:
                break
        
        # Randomly select a pair to connect
        a, b = random.choice(non_adjacent_pairs)
        
        H.add_edge(a, b)
        print(f"Added edge: ({a}, {b})")
        diameter = H.diameter()
    
    return H

In [14]:
G = graphs.PathGraph(5)
G.show(layout="circular")
print(G.diameter())
H = add_edges_until_diameter_leq_2(G, seed=42)
print(H.diameter())

H.show(layout="circular")

Graphics object consisting of 10 graphics primitives
4
Added edge: (0, 2)
Added edge: (1, 3)
Added edge: (0, 3)
2
Graphics object consisting of 13 graphics primitives


We will use this function to generate random graphs that are candidates for the `alpha_od(G)` = 1.

In [15]:
def generate_large_graphs_with_alpha_od_equal_to_1(n, attempts, seed=None):
    """
    Generate 'count' random graphs with 'n' vertices and diameter <= 2,
    and return those graphs G for which alpha_od(G) == 1.
    """
    import random
    if seed is not None:
        random.seed(int(seed))
    
    matches = list()
    seen_isomorphs = set()
    
    for _ in range(attempts):
        G = graphs.RandomGNP(n, p=0.5, seed=random.randint(0, int(1e6)))
        H = add_edges_until_diameter_leq_2(G, seed=random.randint(0, int(1e6)))
        
        canon_repr = H.canonical_label().graph6_string()
        if canon_repr in seen_isomorphs:
            continue # dont compute alpha_od for already seen graphs
        
        if alpha_od(H) == 1:
            seen_isomorphs.add(canon_repr)
            matches.append(H)
            
    return matches

In [16]:
for G in generate_large_graphs_with_alpha_od_equal_to_1(30, 5, seed=123):
    G.show(layout="circular")

Graphics object consisting of 245 graphics primitives
Graphics object consisting of 229 graphics primitives
Graphics object consisting of 259 graphics primitives
Graphics object consisting of 258 graphics primitives
Graphics object consisting of 244 graphics primitives


Testiranje hipotez

In [27]:
from itertools import combinations
from sage.graphs.graph_generators import graphs

def connected_graphs_upto(n_max):
    """
    Generator vseh povezanih, po izomorfizmu različnih grafov
    z n vozlišči, 1 <= n <= n_max.
    """
    for n in range(1, n_max + 1):
        for G in graphs.nauty_geng(f"{n} -c"):
            yield G

def counterexamples_necessity(prop, n_max):
    """
    NUJNOST: isce grafe z alpha_od(G) = 1, ki NE zadoščajo prop(G).

    Če vrne prazen seznam, velja
       alpha_od(G) = 1  ⇒  prop(G)
    za vse povezane grafe z največ n_max vozlišči.
    """
    bad = []
    for G in connected_graphs_upto(n_max):
        # opcijsko: pogoj diam(G) <= 2 pospeši, če je nujen
        if G.diameter() > 2:
            continue
        if alpha_od(G) == 1 and not prop(G):
            bad.append(G)
    return bad


def counterexamples_sufficiency(prop, n_max):
    """
    ZADOSTNOST: isce grafe, ki zadoščajo prop(G), a imajo alpha_od(G) != 1.

    Če vrne prazen seznam, velja
       prop(G)  ⇒  alpha_od(G) = 1
    za vse povezane grafe z največ n_max vozlišči.
    """
    bad = []
    for G in connected_graphs_upto(n_max):
        if prop(G) and alpha_od(G) != 1:
            bad.append(G)
    return bad

def stats_alpha_od_1_and_property(prop, n_max):
    """
    Vrne slovar:
      n ↦ (št. grafov z alpha_od(G)=1, št. tistih med njimi z lastnostjo prop)
    """
    stats = {}
    for n in range(1, n_max + 1):
        total_alpha1 = 0
        total_alpha1_P = 0
        for G in graphs.nauty_geng(f"{n} -c"):
            if G.diameter() > 2:
                continue
            if alpha_od(G) == 1:
                total_alpha1 += 1
                if prop(G):
                    total_alpha1_P += 1
        stats[n] = (total_alpha1, total_alpha1_P)
    return stats


def prop_H1_alpha_square_le_2(G):
    """
    H1: alpha(G^2) <= 2, tj. v G^2 ni neodvisne trojke.
    To izvedemo tako, da preverimo vsak trojček (a,b,c):
    v G^2 ne sme biti trojček, kjer med nobenim parom ni roba.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    
    def adj_in_square(x, y):
        # povezava v G^2 ⇔ razdalja <= 2 in x != y
        if x == y:
            return False
        return dist[x][y] <= 2

    for a, b, c in combinations(V, 3):
        # preverimo, ali so a,b,c paroma nepovezani v G^2
        if (not adj_in_square(a, b) and
            not adj_in_square(a, c) and
            not adj_in_square(b, c)):
            # našli smo neodvisno trojko v G^2
            return False
    return True

def prop_H2_dist2_has_common_neighbor(G):
    """
    H2: Za vsak par u,v z dist(u,v)=2 je |N(u) ∩ N(v)| >= 1.
    """
    dist = G.distance_all_pairs()
    V = list(G.vertices())
    for i in range(len(V)):
        for j in range(i+1, len(V)):
            u, v = V[i], V[j]
            if dist[u][v] == 2:
                common = set(G.neighbors(u)).intersection(G.neighbors(v))
                if len(common) == 0:
                    return False
    return True

properties = ({
    "H1": prop_H1_alpha_square_le_2,
    "H2": prop_H2_dist2_has_common_neighbor,
})

def run_all_tests(n_max=9):
    """
    Za vsako hipotezo izpiše:
      - št. kontra-primerov za nujnost,
      - št. kontra-primerov za zadostnost,
      - statistiko (koliko grafov z alpha_od=1 ima lastnost).
    """
    for name, prop in properties.items():
        print("="*70)
        print(f"Lastnost: {name}")
        print("-"*70)

        bad_nec = counterexamples_necessity(prop, n_max)
        bad_suf = counterexamples_sufficiency(prop, n_max)
        stats = stats_alpha_od_1_and_property(prop, n_max)

        print(f"Nujnost: št. kontra-primerov = {len(bad_nec)}")
        print(f"Zadostnost: št. kontra-primerov = {len(bad_suf)}")
        print("Statistika po n (n: alpha_od=1 grafov, od teh z lastnostjo P):")
        for n in sorted(stats.keys()):
            t1, t2 = stats[n]
            print(f"  n={n}: {t1}  /  {t2}")
        print()


In [18]:
run_all_tests(8)

Lastnost: H1
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 8208
Statistika po n (n: alpha_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11
  n=6: 43  /  43
  n=7: 266  /  266
  n=8: 3042  /  3042

Lastnost: H2
----------------------------------------------------------------------
Nujnost: št. kontra-primerov = 0
Zadostnost: št. kontra-primerov = 8743
Statistika po n (n: alpha_od=1 grafov, od teh z lastnostjo P):
  n=1: 1  /  1
  n=2: 1  /  1
  n=3: 2  /  2
  n=4: 4  /  4
  n=5: 11  /  11
  n=6: 43  /  43
  n=7: 266  /  266
  n=8: 3042  /  3042



Testiranje za grafe z višjim številom vozlišč.

In [25]:
def test_H1_H2_on_generated_alpha_od_1_graphs(
    n,
    attempts=1000,
    seed=None,
    max_keep=None,
    verbose=True,
):
    """
    Uporabi generator generate_large_graphs_with_alpha_od_equal_to_1
    in za vrnjene grafe (alpha_od(G)=1, diam<=2, brez izomorfnih duplikatov)
    preveri lastnosti H1 in H2.

    """
    # 1) generiraj grafe z alpha_od(G) = 1
    graphs_alpha1 = generate_large_graphs_with_alpha_od_equal_to_1(
        n=n,
        attempts=attempts,
        seed=seed,
    )

    # po potrebi omejimo na max_keep
    if max_keep is not None and len(graphs_alpha1) > max_keep:
        graphs_alpha1 = graphs_alpha1[:max_keep]

    tested_graphs = []
    bad_H1 = []
    bad_H2 = []

    # 2) testiraj H1 in H2 na vsakem takem grafu
    for G in graphs_alpha1:
        tested_graphs.append(G)

        if not prop_H1_alpha_square_le_2(G):
            bad_H1.append(G)
        if not prop_H2_dist2_has_common_neighbor(G):
            bad_H2.append(G)

    if verbose:
        print(f"n = {n}")
        print(f"  Št. grafov z alpha_od(G)=1, ki jih je generator vrnil: {len(graphs_alpha1)}")
        print(f"  Št. testiranih grafov: {len(tested_graphs)}")
        print(f"  Št. teh grafov, ki kršijo H1: {len(bad_H1)}")
        print(f"  Št. teh grafov, ki kršijo H2: {len(bad_H2)}")

    return bad_H1, bad_H2, tested_graphs

def scan_H1_H2_on_generated_alpha_od_1_graphs(
    n_values,
    attempts=5000,
    seed=None,
    max_keep=200,
):
    """
    Za vsak n v n_values:
      - uporabi generator za grafe z alpha_od(G)=1,
      - testiraj H1 in H2,
      - shrani kratko statistiko v slovar.

    Vrne: dict n -> {"num_tested", "num_bad_R2", "num_bad_N4"}.
    """
    results = {}
    for n in n_values:
        print("="*60)
        print(f"Testiranje za n = {n}")
        bad_H1, bad_H2, tested = test_H1_H2_on_generated_alpha_od_1_graphs(
            n=n,
            attempts=attempts,
            seed=seed,
            max_keep=max_keep,
            verbose=True,
        )
        results[n] = {
            "num_tested": len(tested),
            "num_bad_R2": len(bad_H1),
            "num_bad_N4": len(bad_H2),
        }
    return results


In [24]:
res = scan_H1_H2_on_generated_alpha_od_1_graphs(
    n_values=[20],
    attempts=10000,
    seed=42,
    max_keep=300,
)
res

Testiranje za n = 20
Added edge: (2, 0)
Added edge: (2, 8)
Added edge: (0, 16)
Added edge: (14, 3)
Added edge: (7, 17)
Added edge: (1, 16)
Added edge: (3, 8)
Added edge: (1, 14)
Added edge: (0, 17)
Added edge: (2, 16)
Added edge: (1, 17)
Added edge: (15, 17)
Added edge: (8, 18)
Added edge: (0, 16)
Added edge: (2, 14)
Added edge: (9, 10)
Added edge: (16, 12)
Added edge: (0, 19)
Added edge: (16, 17)
Added edge: (0, 12)
Added edge: (3, 12)
Added edge: (6, 2)
Added edge: (16, 14)
Added edge: (4, 7)
Added edge: (16, 13)
Added edge: (2, 8)
Added edge: (4, 17)
Added edge: (1, 4)
Added edge: (4, 19)
Added edge: (2, 17)
Added edge: (7, 3)
Added edge: (19, 10)
Added edge: (16, 14)
Added edge: (3, 17)
Added edge: (6, 2)
Added edge: (6, 16)
Added edge: (2, 16)
Added edge: (5, 2)
Added edge: (7, 9)
Added edge: (0, 15)
Added edge: (18, 12)
Added edge: (6, 1)
Added edge: (1, 6)
Added edge: (5, 16)
Added edge: (8, 10)
Added edge: (3, 16)
Added edge: (10, 13)
Added edge: (2, 10)
Added edge: (1, 6)
Adde

{20: {'num_tested': 300, 'num_bad_R2': 0, 'num_bad_N4': 0}}